In [58]:
# -*- coding: utf-8 -*-
"""
Created on Fri 21 2020
@author: Thiago Pinho
"""

# -*- coding: utf-8 -*-
"""
Created on Fri 21 2020
@author: Thiago Pinho
"""

from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np
import spacy

VECTOR_MODEL_NAME = 'pt_core_news_sm'
RELATIVE_PATH_TO_CSV = "./assets/datasets/ribon/Feeds_Label.csv"

# load the dataset
df_ribon_news = pd.read_csv(RELATIVE_PATH_TO_CSV)
print(df_ribon_news.head())

   id  pick_count                                            content  \
0  11           0  A forte ligação dos cães com o ser humano desp...   
1  12           0  Os chamados muros da gentileza estão promovend...   
2   9           0  Alguns de nós sabem a sorte que é ter um irmão...   
3   6           0               Temos muito o que aprender com elas.   
4   1           0  Esqueça o que te disseram sobre a zona de conf...   

                                               title    Label_Train  
0  Há mais de 10 anos, dupla de cães 🐶 acompanha ...        Animais  
1  Muros da gentileza ajudam desabrigados e famíl...  Solidariedade  
2  20 fotos maravilhosas mostrando a alegria de t...        Familia  
3  13 razões pelas quais crianças são bem mais es...       Criancas  
4  Ex-executiva fala sobre vida sem propósito - I...      Proposito  


In [59]:
df_ribon_news.columns = map(lambda x: str(x).upper(), df_ribon_news.columns)
df_ribon_news['LABEL_TRAIN'] = df_ribon_news['LABEL_TRAIN'].str.upper()
''' Converting all labels to uppercase '''
print(df_ribon_news.head())

   ID  PICK_COUNT                                            CONTENT  \
0  11           0  A forte ligação dos cães com o ser humano desp...   
1  12           0  Os chamados muros da gentileza estão promovend...   
2   9           0  Alguns de nós sabem a sorte que é ter um irmão...   
3   6           0               Temos muito o que aprender com elas.   
4   1           0  Esqueça o que te disseram sobre a zona de conf...   

                                               TITLE    LABEL_TRAIN  
0  Há mais de 10 anos, dupla de cães 🐶 acompanha ...        ANIMAIS  
1  Muros da gentileza ajudam desabrigados e famíl...  SOLIDARIEDADE  
2  20 fotos maravilhosas mostrando a alegria de t...        FAMILIA  
3  13 razões pelas quais crianças são bem mais es...       CRIANCAS  
4  Ex-executiva fala sobre vida sem propósito - I...      PROPOSITO  


In [60]:
# Viewing frequencies
for label in df_ribon_news['LABEL_TRAIN'].unique():
    print(label + ": ", len(df_ribon_news[df_ribon_news.LABEL_TRAIN == label]))


ANIMAIS:  67
SOLIDARIEDADE:  94
FAMILIA:  19
CRIANCAS:  19
PROPOSITO:  86
DIVERSIDADE:  72
EDUCACAO:  66
ECOLOGIA:  129
SAUDE:  24
IDOSOS:  15
CIENCIA:  31
CELEBRACAO:  56
ETNIAS:  65
ESPORTE:  34
CULTURA:  79
EMPREENDEDORISMO:  32


In [62]:
# For simple label encoding
# All categorical columns
categorical_cols = ['LABEL_TRAIN']

label_df_ribon_news = df_ribon_news

# Apply label encoder
label_encoder = LabelEncoder()
for col in categorical_cols:
    label_df_ribon_news[col] = label_encoder.fit_transform(df_ribon_news[col])

print(label_df_ribon_news.head())

   ID  PICK_COUNT                                            CONTENT  \
0  11           0  A forte ligação dos cães com o ser humano desp...   
1  12           0  Os chamados muros da gentileza estão promovend...   
2   9           0  Alguns de nós sabem a sorte que é ter um irmão...   
3   6           0               Temos muito o que aprender com elas.   
4   1           0  Esqueça o que te disseram sobre a zona de conf...   

                                               TITLE  LABEL_TRAIN  
0  Há mais de 10 anos, dupla de cães 🐶 acompanha ...            0  
1  Muros da gentileza ajudam desabrigados e famíl...           15  
2  20 fotos maravilhosas mostrando a alegria de t...           11  
3  13 razões pelas quais crianças são bem mais es...            3  
4  Ex-executiva fala sobre vida sem propósito - I...           13  


In [56]:
# Load the large model to get the vectors
nlp = spacy.load(VECTOR_MODEL_NAME)

# We just want the vectors so we can turn off other models in the pipeline
with nlp.disable_pipes():
    vectors = np.array([nlp(str(news.CONTENT)).vector for idx, news in df_ribon_news.iterrows()])
    
vectors.shape


(888, 96)

In [63]:
# Training models

X_train, X_test, y_train, y_test = train_test_split(vectors, label_df_ribon_news.LABEL_TRAIN, 
                                                    test_size=0.1, random_state=1)

# Create the LinearSVC model
model = LinearSVC(random_state=1, dual=False)
#Fit the model
model.fit(X_train, y_train)

# Uncomment and run to see model accuracy
print(f'Model test accuracy: {model.score(X_test, y_test)*100:.3f}%')

Model test accuracy: 16.854%


In [64]:
# Scratch space in case you want to experiment with other models

second_model = RandomForestRegressor(random_state=2)
second_model.fit(X_train, y_train)
print(f'Model test accuracy: {second_model.score(X_test, y_test)*100:.3f}%')

Model test accuracy: 10.566%
